<a href="https://colab.research.google.com/github/shaoming/financial_tool/blob/master/stock_options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

track options to predict stock price

In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install swifter
!pip install pandas

In [2]:
from google.auth import default
import gspread
creds, _ = default()
gc = gspread.authorize(creds)

import yfinance as yf
from yahooquery import Ticker
from google.colab import data_table
from IPython.display import display, HTML

In [3]:
def getTopOpenInterest(options_chain):
  options_chain.reset_index()
  return options_chain.sort_values('volume', ascending=False).nlargest(20, 'openInterest').sort_values('expiration', ascending=True)

In [4]:
def getDerivedOfOption(df):
  result = df
  option_group = result.sort_values(['strike','expiration','optionType'],ascending=True)
  option_group["price"] = option_group["bid"] 
  option_group["previous_strike"] = option_group.groupby(['expiration','optionType'])['strike'].shift()
  option_group["previous_lastPrice"] = option_group.groupby(['expiration','optionType'])['price'].shift()
  option_group = option_group.dropna()
  option_group = option_group.reset_index()

  option_group = option_group.loc[(option_group['symbol'].shift() == option_group['symbol']) & (option_group['expiration'].shift() == option_group['expiration']) & (option_group['optionType'].shift() == option_group['optionType'])]
  option_group["previous_strike"] = option_group["strike"] - option_group["previous_strike"]#option_group["previous_close"]
  option_group["previous_lastPrice"] = option_group["price"] - option_group["previous_lastPrice"]
  option_group["DerivedProb"] = option_group["previous_lastPrice"] / option_group["previous_strike"]
  option_group = option_group.sort_values(['strike','expiration','optionType'],ascending=False)
  option_group = option_group.loc[(option_group["DerivedProb"] > 0.1) | (option_group["DerivedProb"] < -0.1)]
  DerivedProb = option_group.pop('DerivedProb')
  option_group.insert(1, 'DerivedProb', DerivedProb)

  return option_group

In [5]:
def getStddev(etf):
  import numpy as np
  Data = Ticker(etf).history(period='5mo', interval='5d')
  return np.std(Data["close"].tolist())




In [ ]:
from scipy.stats import norm
norm.expect(lambda x: x-65 if x>65 else 0, loc=90, scale=1, lb=0,ub=100, conditional=True)

In [17]:

def getMeanValue(stddev, strike_price, option_value, optionType, minValue, maxValue):
  from scipy.stats import norm
  import numpy as np
  target = np.nan
  distance = np.finfo(np.float32).max
  if optionType =="calls":
         for mean_value in np.arange(maxValue, minValue, -1):
           expect_value = norm.expect(lambda x: x-strike_price if x> strike_price else 0,loc=mean_value, scale=stddev, lb=minValue,ub=maxValue)
           if abs(option_value - expect_value) < distance:
             distance = abs(option_value - expect_value)
             target = mean_value  
  else:
         for mean_value in np.arange(minValue, maxValue, 1):    
            expect_value = norm.expect(lambda x: strike_price - x if x< strike_price else 0,loc=mean_value, scale=stddev, lb=minValue,ub=maxValue)
            if abs(option_value - expect_value) < distance:
              distance = abs(option_value - expect_value)
              target = mean_value
  return target

def getExpectedValues(etf, df, minValue, maxValue):
  import swifter
  
  stddev = getStddev(etf)
  data = df.reset_index()

  data.loc[:,'expected_mean'] = data.swifter.apply(lambda x:getMeanValue(stddev, x["strike"],x["lastPrice"],x["optionType"], minValue, maxValue), axis=1)
  expected_mean = data.pop('expected_mean')
  data.insert(1, 'expected_mean', expected_mean)
  return data

def getMeanExpectedValues(result, etf_name):
  result = result.reset_index()
  result = result.dropna()
  result = result[['expiration','optionType', 'inTheMoney','volume','expected_mean']]
  result.loc[:,'expected_mean_volume'] = result['volume']*result['expected_mean']

  result=result.groupby(['expiration','optionType', 'inTheMoney']).sum()
  result.loc[:,'avg_expected_mean'] = result['expected_mean_volume'] / result['volume']
  result = result.reset_index()
  #print(result)
  import matplotlib.pyplot as plt
  from datetime import datetime
  currentDateAndTime = datetime.now() 
  plt.rcParams['figure.figsize'] = [10, 4]
  plt.plot('expiration', 'avg_expected_mean', data=result.loc[(result['optionType']=="calls")&(result['inTheMoney']==True)], marker='', color='blue', linewidth=2, linestyle='dashed', label="Call&InMoney")
  plt.plot('expiration', 'avg_expected_mean', data=result.loc[(result['optionType']=="calls")&(result['inTheMoney']==False)], marker='', color='green', linewidth=2, linestyle='dashed', label="Call&NotInMoney")
  plt.plot('expiration', 'avg_expected_mean', data=result.loc[(result['optionType']=="puts")&(result['inTheMoney']==True)], marker='', color='orange', linewidth=2, linestyle='dashed', label="Put&InMoney")
  plt.plot('expiration', 'avg_expected_mean', data=result.loc[(result['optionType']=="puts")&(result['inTheMoney']==False)], marker='', color='red', linewidth=2, linestyle='dashed', label="Put&NotInMoney")
  plt.title("option expectation: --"+etf_name+"-- "+currentDateAndTime.strftime("%a, %d %b %Y %I:%M:%S"))
  plt.legend()
  plt.show()

In [8]:
def getMinMaxValue(df):
  import pandas as np
  maxValue = df["strike"].max()
  minValue = df["strike"].min()
  return minValue, maxValue

def filterData(df):
  import pandas as pd
  data = df.loc[(df['bid']> 0.05)&(df['openInterest']>100)] ## remove trivial number
  data["lastTradeDate"] = pd.to_datetime(data['lastTradeDate'], format='%Y-%m-%d %H:%M:%S')
  #df = df.loc[(df["lastTradeDate"] > pd.to_datetime(yesterday))]
  return data

In [ ]:
#ETF_list = ['spy','qqq','xlv','xlc','xli','xlb','xlk','xlu','xlf','iwm','xbi','ibb']
ETF_list = ['xbi']
from datetime import datetime
from datetime import date, timedelta

yesterday = date.today() - timedelta(days=3)
import pandas as pd
for etf in ETF_list: 
  df = Ticker(etf).option_chain
  minValue, maxValue = getMinMaxValue(df)
  previous_close = Ticker(etf).summary_detail[etf]['previousClose']
  df.loc[:,'previous_close'] = previous_close
  df = filterData(df)
  df = df.reset_index()
  #df = df.loc[(df['optionType']=='calls') & (df['inTheMoney'] == True)]
  #topOpenInterest = getTopOpenInterest(df)
  
  meanValues = getExpectedValues(etf, df, minValue, maxValue)
  getMeanExpectedValues(meanValues, etf)
  
  #display(HTML(meanValues.to_html()))
